# build a resnet50 to classify image
构建resnet50网络完成图像分类

In [25]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import cv2
from imutils import paths
import random

In [26]:
norm_size = 32
CLASS_NUM = 3
EPOCHS = 35
INIT_LR = 1e-3
BS = 32

In [27]:
#编写加载图像的函数
def load_data(path):
    print("[INFO] loading images...")
    data = []
    labels = []
    # grab the image paths and randomly shuffle them
    imagePaths = sorted(list(paths.list_images(path)))
    random.seed(42)
    random.shuffle(imagePaths)
    # loop over the input images
    for imagePath in imagePaths:
        img = image.load_img(imagePath, target_size=(224, 224))
        x = image.img_to_array(img)
        data.append(x)
        #label是上级目录名
        label = int(imagePath.split(os.path.sep)[-2])       
        labels.append(label)
    # scale the raw pixel intensities to the range [0, 1]
    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)

    # convert the labels from integers to vectors
    labels = to_categorical(labels, num_classes=CLASS_NUM)                         
    return data,labels

In [ ]:
#搭建网络
#加载预训练模型，需要下载预训练模型，可能会报错ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
model = ResNet50(weights='imagenet')
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"])

  1359872/102853048 [..............................] - ETA: 28:55

In [ ]:
train_file_path = r'E:\workplace\Python\Memotion Analysis\Memotion Analysis\train'
test_file_path = r'E:\workplace\Python\Memotion Analysis\Memotion Analysis\trial'
trainX,trainY = load_data(train_file_path)
testX,testY = load_data(test_file_path)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#通过实时数据增强生成张量图像数据批次。数据将不断循环（按批次）。
#rotation_range: 整数。随机旋转的度数范围。
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
        horizontal_flip=True, fill_mode="nearest")

model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
        validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
        epochs=EPOCHS, verbose=1)